<a href="https://colab.research.google.com/github/vishnoiprem/Chargebee-etl/blob/master/kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print(1)

1


In [ ]:
#This is user for set up kafka and colab becoz it is easy to use


In [ ]:
#Install the required tensorflow-io and kafka packages

[link text](https://)

In [ ]:
!pip install kafka-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)


In [ ]:


# This is formatted as code

### Import packages


In [ ]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
import pandas as pd

## Download and setup Kafka and Zookeeper instances

For demo purposes, the following instances are setup locally:

- Kafka (Brokers: 127.0.0.1:9092)
- Zookeeper (Node: 127.0.0.1:2181)


In [ ]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.1.0/kafka_2.13-3.1.0.tgz
!tar -xzf kafka_2.13-3.1.0.tgz

*Using* the default configurations (provided by Apache Kafka) for spinning up the instances.

In [ ]:
!./kafka_2.13-3.1.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.1.0/config/zookeeper.properties
!./kafka_2.13-3.1.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.1.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


**Once** the instances are started as daemon processes, grep for `kafka` in the processes list. The two java processes correspond to zookeeper and the kafka instances. *italicised text*

In [ ]:
!ps -ef | grep kafka

root         580       1  1 13:47 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-3.1.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-3.1.0/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-3.1.0/bin/../config/log4j.properties -cp /content/kafka_2.13-3.1.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-3.1.0/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-3.1.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-3.1.0/bin/../libs/audience-annotations-0.5.0.jar:/content/kafka_2.13-3.1.0/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-3.1.0/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.13

Create the kafka topics with the following specs:

rejected_2007_to_2018Q4: partitions=1, replication-factor=1


In [ ]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic rejected_2007


Created topic rejected_2007.


Describe the topic for details on the configuration

In [ ]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic rejected_2007


Topic: rejected_2007	TopicId: Mw_PX0_oTj2xJc4kGdrpxg	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: rejected_2007	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


## kaggle Dataset

Kafka being an event streaming platform, enables  data from various sources to be written into it. For instance:

- Web traffic logs
- Astronomical measurements
- IoT sensor data
- Product reviews and many more.

For the purpose of this tutorial, lets download the [SUSY](https://archive.ics.uci.edu/ml/datasets/SUSY#) dataset and feed the data into kafka manually. The goal of this classification problem is to distinguish between a signal process which produces supersymmetric particles and a background process which does not.


In [ ]:
#!curl -sSOL https://www.kaggle.com/datasets/wordsforthewise/lending-club?select=rejected_2007_to_2018Q4.csv.gz


!curl -sSOL  "https://www.kaggle.com/datasets/wordsforthewise/lending-club?select=rejected_2007_to_2018Q4.csv.gz" 

In [ ]:


curl -L https://protonmail.com/download/bridge/protonmail-bridge_2.1.1-1_amd64.deb -o bridge.deb



susy_iterator = pd.read_csv("'lending-club?select=rejected_2007_to_2018Q4.csv.gz'",compression='gzip')
susy_iterator.head()



FileNotFoundError: ignored

In [ ]:
!wget https://www.kaggle.com/datasets/wordsforthewise/lending-club?select=rejected_2007_to_2018Q4.csv.gz

--2022-05-30 14:55:02--  https://www.kaggle.com/datasets/wordsforthewise/lending-club?select=rejected_2007_to_2018Q4.csv.gz
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘lending-club?select=rejected_2007_to_2018Q4.csv.gz.1’

lending-club?select     [ <=>                ]  19.87K  --.-KB/s    in 0.01s   

2022-05-30 14:55:02 (1.55 MB/s) - ‘lending-club?select=rejected_2007_to_2018Q4.csv.gz.1’ saved [20346]



In [ ]:
!ls

 kafka_2.13-3.1.0
 kafka_2.13-3.1.0.tgz
'lending-club?select=rejected_2007_to_2018Q4.csv.gz'
'lending-club?select=rejected_2007_to_2018Q4.csv.gz.1'
 sample_data


In [ ]:
!ls -lrt

total 86096
drwxr-xr-x 1 root root     4096 May 17 13:39  sample_data
-rw-r--r-- 1 root root 88130011 May 30 13:46  kafka_2.13-3.1.0.tgz
drwxr-xr-x 8 root root     4096 May 30 13:47  kafka_2.13-3.1.0
-rw-r--r-- 1 root root    20286 May 30 15:03 'lending-club?select=rejected_2007_to_2018Q4.csv.gz'


In [ ]:
!unzip lending-club?select=rejected_2007_to_2018Q4.csv.gz'

/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
!ls -lrt

total 86096
drwxr-xr-x 1 root root     4096 May 17 13:39  sample_data
-rw-r--r-- 1 root root 88130011 May 30 13:46  kafka_2.13-3.1.0.tgz
drwxr-xr-x 8 root root     4096 May 30 13:47  kafka_2.13-3.1.0
-rw-r--r-- 1 root root    20286 May 30 15:03 'lending-club?select=rejected_2007_to_2018Q4.csv.gz'


# Newhttps://www.kaggle.com/general/74235 section

In [ ]:
from google.colab import files
files.upload()         # expire any previous token(s) and upload recreated token


In [ ]:
# !rm -r ~/.kaggle
# !mkdir ~/.kaggle
# !mv ./kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets list